# 3.0 – Solar Power Prediction · Upgraded Bidirectional GRU

**Fixes from `2.0-JD-energy_load_prediction.ipynb`:**
1. Model is now truly **Bidirectional GRU** (not plain GRU)
2. `drop()` is now performed **inplace** – no more data leakage from `Predicted Load (kW)`
3. **Separate scaler** for the Solar Power target
4. Window increased from 24 → **96 steps (24 hours)**
5. **Cyclic sin/cos encoding** for Hour and Day-of-Year
6. **Single-target** model (Solar Power only) – no dilution
7. **Month feature** added for seasonality
8. Better hyperparameters (dropout=0.25, batch=64, Huber loss)

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

from tensorflow.keras.models import Sequential                    # type: ignore
from tensorflow.keras.layers import (                             # type: ignore
    Input, Dense, GRU, Dropout, BatchNormalization, Bidirectional
)
from tensorflow.keras.optimizers import Adam                      # type: ignore
from tensorflow.keras.regularizers import l2                      # type: ignore
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau  # type: ignore

print(f"Pandas   : {pd.__version__}")
print(f"Sklearn  : {sklearn.__version__}")
print(f"TF       : {tf.__version__}")

In [ ]:
RAW_DATA_PATH    = '../data/raw/dataset.csv'
TRAIN_DATA_PATH  = '../data/processed/train/dataset.csv'
TEST_DATA_PATH   = '../data/processed/test/dataset.csv'
MODEL_DIR        = '../models/'
REPORT_DIR       = '../reports/figures/'
os.makedirs(REPORT_DIR, exist_ok=True)

## 1 · Load & Clean Dataset

In [ ]:
df = pd.read_csv(RAW_DATA_PATH, parse_dates=['Timestamp'])
df.set_index('Timestamp', inplace=True)
df.sort_index(inplace=True)
df = df[~df.index.duplicated(keep='first')]
print(f"Dataset shape: {df.shape}")
print(f"Date range  : {df.index.min()} → {df.index.max()}")
df.head(3)

In [ ]:
# FIX #2 – Drop leaky / irrelevant columns INPLACE
# 'Predicted Load (kW)' leaks target info; 'Transformer Fault' is a rare flag
LEAKY_COLS = ['Predicted Load (kW)', 'Transformer Fault']
df.drop(columns=LEAKY_COLS, inplace=True)
print("Remaining columns:", df.columns.tolist())

## 2 · Feature Engineering

In [ ]:
# FIX #5 – Cyclic sin/cos encoding for time (replaces raw integers)
# FIX #7 – Add Month for seasonality

df['hour_sin']  = np.sin(2 * np.pi * df.index.hour / 24)
df['hour_cos']  = np.cos(2 * np.pi * df.index.hour / 24)

df['dow_sin']   = np.sin(2 * np.pi * df.index.dayofweek / 7)
df['dow_cos']   = np.cos(2 * np.pi * df.index.dayofweek / 7)

df['doy_sin']   = np.sin(2 * np.pi * df.index.dayofyear / 365)
df['doy_cos']   = np.cos(2 * np.pi * df.index.dayofyear / 365)

df['month_sin'] = np.sin(2 * np.pi * df.index.month / 12)
df['month_cos'] = np.cos(2 * np.pi * df.index.month / 12)

df['is_weekend'] = df.index.dayofweek.isin([5, 6]).astype(int)

print(f"Feature count after engineering: {df.shape[1]}")
df.head(2)

## 3 · Train / Test Split (70 / 15 / 15)

In [ ]:
n = len(df)
train_end = int(n * 0.70)
test_end  = int(n * 0.85)

train_df = df.iloc[:train_end].copy()
test_df  = df.iloc[train_end:test_end].copy()
live_df  = df.iloc[test_end:].copy()

print(f"Train : {len(train_df):>6} rows  {train_df.index.min()} → {train_df.index.max()}")
print(f"Test  : {len(test_df):>6} rows  {test_df.index.min()} → {test_df.index.max()}")
print(f"Live  : {len(live_df):>6} rows  {live_df.index.min()} → {live_df.index.max()}")

## 4 · Scaling

**FIX #3** – Use **two separate scalers**:
- `feature_scaler` : for input features
- `target_scaler`  : for `Solar Power (kW)` target only → enables clean inverse-transform

In [ ]:
TARGET_COL = 'Solar Power (kW)'

# Columns to scale (continuous; not already cyclic/binary)
SCALE_COLS = [
    'Voltage (V)', 'Current (A)', 'Power Consumption (kW)',
    'Reactive Power (kVAR)', 'Power Factor',
    'Solar Power (kW)', 'Wind Power (kW)',
    'Grid Supply (kW)', 'Voltage Fluctuation (%)',
    'Temperature (°C)', 'Humidity (%)',
    'Electricity Price (USD/kWh)'
]

feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler  = MinMaxScaler(feature_range=(0, 1))

# Fit scalers on training data only (prevents leakage)
train_df[SCALE_COLS] = feature_scaler.fit_transform(train_df[SCALE_COLS])
test_df[SCALE_COLS]  = feature_scaler.transform(test_df[SCALE_COLS])

# Separate target scaler
target_scaler.fit(train_df[[TARGET_COL]])

# Save scalers
joblib.dump(feature_scaler, os.path.join(MODEL_DIR, 'scaler/solar_feature_scaler.pkl'))
joblib.dump(target_scaler,  os.path.join(MODEL_DIR, 'scaler/solar_target_scaler.pkl'))
print("Scalers saved.")

## 5 · Sequence Generation

**FIX #4** – Window = **96 steps = 24 hours** (was 24 steps = 6 hours, missing full daily solar cycle)

In [ ]:
TARGET_IDX = train_df.columns.get_loc(TARGET_COL)
WINDOW = 96   # 96 × 15 min = 24 hours

def make_sequences(data: pd.DataFrame, target_idx: int, window: int):
    """Create (X, y) sequences for a univariate target from a multi-feature DataFrame."""
    arr = data.values.astype('float32')
    X, y = [], []
    for i in range(len(arr) - window):
        X.append(arr[i : i + window])          # shape (window, features)
        y.append(arr[i + window, target_idx])  # scalar
    return np.array(X), np.array(y).reshape(-1, 1)

X_train, y_train = make_sequences(train_df, TARGET_IDX, WINDOW)
X_test,  y_test  = make_sequences(test_df,  TARGET_IDX, WINDOW)

print(f"X_train : {X_train.shape}  y_train : {y_train.shape}")
print(f"X_test  : {X_test.shape}   y_test  : {y_test.shape}")

## 6 · Model Architecture – True Bidirectional GRU

**FIX #1** – Every recurrent layer is wrapped with `Bidirectional()`. The original model had plain `GRU()` layers.

In [ ]:
STEPS     = X_train.shape[1]   # 96
FEATURES  = X_train.shape[2]
DROP      = 0.25               # was 0.1 – too low
REG       = 1e-4

model = Sequential([
    Input(shape=(STEPS, FEATURES)),

    # --- Bidirectional GRU Block 1 ---
    Bidirectional(
        GRU(128, activation='tanh', recurrent_activation='sigmoid',
            kernel_regularizer=l2(REG),
            return_sequences=True)
    ),
    BatchNormalization(),
    Dropout(DROP),

    # --- Bidirectional GRU Block 2 ---
    Bidirectional(
        GRU(64, activation='tanh', recurrent_activation='sigmoid',
            kernel_regularizer=l2(REG),
            return_sequences=True)
    ),
    BatchNormalization(),
    Dropout(DROP),

    # --- Bidirectional GRU Block 3 ---
    Bidirectional(
        GRU(32, activation='tanh', recurrent_activation='sigmoid',
            kernel_regularizer=l2(REG))
    ),
    BatchNormalization(),
    Dropout(DROP),

    # --- Dense Head ---
    Dense(64, activation='relu', kernel_initializer='he_uniform'),
    Dropout(DROP / 2),
    Dense(32, activation='relu', kernel_initializer='he_uniform'),
    Dense(1)   # Single-target: Solar Power (kW)
], name='BiGRU_SolarPower')

model.summary()

## 7 · Compile & Train

In [ ]:
OPTIMIZER = Adam(learning_rate=1e-3)

model.compile(
    optimizer=OPTIMIZER,
    loss='huber',          # robust to outliers (replaces log_cosh)
    metrics=['mae', 'mse', tf.keras.metrics.RootMeanSquaredError(name='rmse')]
)

EARLY_STOP   = EarlyStopping(
    monitor='val_loss', patience=15,
    restore_best_weights=True, verbose=1
)
LR_SCHEDULER = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5,   # was 0.2 – too aggressive
    patience=7, min_lr=1e-6, verbose=1
)

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,          # was 16 – slow & noisy for 35K samples
    validation_data=(X_test, y_test),
    callbacks=[EARLY_STOP, LR_SCHEDULER],
    verbose=1
)

## 8 · Evaluate & Save

In [ ]:
SOLAR_MODEL_PATH = os.path.join(MODEL_DIR, 'core/solar_bigru.keras')
model.save(SOLAR_MODEL_PATH)
print(f"Model saved → {SOLAR_MODEL_PATH}")

In [ ]:
# Load model back and predict
loaded_model = tf.keras.models.load_model(SOLAR_MODEL_PATH)
PREDICTIONS_SCALED = loaded_model.predict(X_test)
print(f"Predictions shape: {PREDICTIONS_SCALED.shape}")

In [ ]:
# FIX #3 – Clean inverse-transform with dedicated target scaler
PRED = target_scaler.inverse_transform(PREDICTIONS_SCALED)
TRUE = target_scaler.inverse_transform(y_test)

mae  = mean_absolute_error(TRUE, PRED)
rmse = np.sqrt(mean_squared_error(TRUE, PRED))
mape = np.mean(np.abs((TRUE - PRED) / (TRUE + 1e-5))) * 100  # avoid div-by-zero at night

print(f"{'='*45}")
print(f"  Solar Power (kW) – Test Set Results")
print(f"{'='*45}")
print(f"  MAE  : {mae:.3f} kW")
print(f"  RMSE : {rmse:.3f} kW")
print(f"  MAPE : {mape:.2f}%")
print(f"{'='*45}")

## 9 · Plots

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(history.history['loss'],     label='Train Loss')
axes[0].plot(history.history['val_loss'], label='Val Loss')
axes[0].set_title('Huber Loss')
axes[0].set_xlabel('Epoch')
axes[0].legend()

axes[1].plot(history.history['mae'],     label='Train MAE')
axes[1].plot(history.history['val_mae'], label='Val MAE')
axes[1].set_title('Mean Absolute Error')
axes[1].set_xlabel('Epoch')
axes[1].legend()

plt.tight_layout()
plt.savefig(os.path.join(REPORT_DIR, 'solar_training_curves.png'), dpi=150)
plt.show()

In [ ]:
# Show 5 days (480 × 15-min = 120 hours) of actual vs predicted
N = 480
fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(TRUE[:N], label='Actual Solar Power', color='#2196F3', linewidth=1.5)
ax.plot(PRED[:N], label='Predicted Solar Power', color='#FF9800', linestyle='--', linewidth=1.5)
ax.fill_between(range(N), TRUE[:N].flatten(), PRED[:N].flatten(),
                alpha=0.15, color='red', label='Error area')
ax.set_title('BiGRU Solar Power Prediction – 5-Day Sample', fontsize=14)
ax.set_ylabel('Solar Power (kW)')
ax.set_xlabel('Time Step (15 min)')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(REPORT_DIR, 'solar_prediction_5days.png'), dpi=150)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(TRUE, PRED, alpha=0.1, s=4, color='steelblue')
lims = [min(TRUE.min(), PRED.min()), max(TRUE.max(), PRED.max())]
ax.plot(lims, lims, 'r--', linewidth=1.5, label='Perfect prediction')
ax.set_xlabel('Actual Solar Power (kW)')
ax.set_ylabel('Predicted Solar Power (kW)')
ax.set_title('Actual vs Predicted – Scatter')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(REPORT_DIR, 'solar_scatter.png'), dpi=150)
plt.show()

## 10 · Summary of Changes vs Original Model

In [ ]:
comparison = pd.DataFrame({
    'Aspect': [
        'Architecture',
        'Data leakage fix',
        'Scaling',
        'Lookback window',
        'Time encoding',
        'Output targets',
        'Loss function',
        'Dropout',
        'Batch size',
    ],
    'Original (2.0)': [
        'Plain GRU (NOT bidirectional)',
        'drop() not inplace – LEAKY',
        'Single scaler, wrong inverse-transform',
        '24 steps = 6 hours',
        'Raw integer hour & weekday',
        '3 outputs (Power, Solar, Wind)',
        'log_cosh',
        '0.10',
        '16',
    ],
    'Upgraded (3.0)': [
        'True Bidirectional GRU (3 layers)',
        'drop(inplace=True) – fixed',
        'Separate feature & target scalers',
        '96 steps = 24 hours',
        'sin/cos cyclic encoding',
        '1 output (Solar Power only)',
        'Huber (robust to outliers)',
        '0.25',
        '64',
    ]
})

print(comparison.to_string(index=False))